# Geoindexing/Spatial Indexing

**Indexing for spatial data. How can we make the managment and retreival of data structures with spatial data more efficient (space and time). We focus here on time.**
* This can be in memory
  * python/javascript
* Or in a database
  * Postgis addon to postgris
  
## Problem setup

* Imagine you have 100000 points in a 2 dimentional space (latitude, longitude). Lets call these points "data"
```python
lng  = np.random.uniform(-180, 180, (100000, 1))
lat  = np.random.uniform( -90,  90, (100000, 1))
data = np.concatenate(lat, lng)
```
* And you are given an extra point. Lets call this point "target"
```python
target = np.random.uniform(-180, 180, 2)
```
* Your task is to find the point from data closest to "target". <strong>How would you do this?</strong>

### The bad way

* You would calculate the euclidian distiance to every point in data and "target" and choose the minimum
* Takes too long. This takes linear time for a search operation relative to number of points in data
* O(n)

### The good way

* Use a spacial index **after projecting the lat/lng to an appropriate projection** or **using appropriate distance measures**
* Takes logarithmic time
* O(log(n))

### Other ways

* Using different geohashing methods one could acheive better performance
  * Uber's h3
  * Google S2


## Practical

### Generate data

In [1]:
import numpy as np

lat  = np.random.uniform( -90,  90, (100000, 1))
lng  = np.random.uniform(-180, 180, (100000, 1))
data = np.concatenate([lat, lng], axis=1)

target = (np.random.uniform(-90, 90), np.random.uniform(-180, 180))

data, target

(array([[-16.06305487, -15.98630798],
        [-10.40165439,  51.82928483],
        [  4.1340833 ,  61.23445928],
        ...,
        [ 15.26745171,  93.36594643],
        [-43.24971876, -97.0005834 ],
        [ -3.86684693, -40.17036311]]),
 (74.41952951779595, 172.80289766480428))

### The bad way

* Assume earths polar coordinate system is flat and use euclidian distance. This leads to errors in the results (nearest points and distance measures)
* O(n) time complexity for querying a point

In [2]:
%%time
# Euclidian distance (without square rooting)
dists = ((data[:,0] - target[0])**2 + \
         (data[:,1] - target[1])**2)

idx   = dists.argmin()
dist  = np.sqrt(dists.min())

CPU times: user 5.09 ms, sys: 2.47 ms, total: 7.56 ms
Wall time: 2.2 ms


In [3]:
# Notice the distance here does not represent the real bearing distance on earth's surface
print('index', idx)
print('distance', dist)
print('point', data[idx])
print()

index 57025
distance 0.7207920347972745
point [ 75.13807259 172.7460029 ]



**Possible solutions:**

* Use a different distance measure (bearing distance on earth's surface)
* Use 3d geocentric coordinates (below) 

### The good way (almost), Spacial Indexing with KDtrees

* [RTree](https://en.wikipedia.org/wiki/R-tree)
* [KDTree](https://en.wikipedia.org/wiki/K-d_tree)

features:
* Space partitioning data structure based on n dimentions (in our case 2)
* Iteratively split the data to two parts along each dimention to create a binary tree
* O(log(n)) time complexity for Queries

Disadvantages:

* **Notice** that using the euclidian distance here would not return accurate results for polar coordinates like longitudes and latitudes!
* When you query a nearest point, you do not get the nearest point after the initial tree depth search, you might need to traverse upwards to correct!
* Does not work well for higher dimentions (curse of dimentionality)

<img src="img/kdtree.png" width="600px">

In [4]:
from scipy.spatial import KDTree

In [5]:
%%time
tree = KDTree(data)

CPU times: user 35.5 ms, sys: 3.15 ms, total: 38.7 ms
Wall time: 37.6 ms


In [6]:
%%time
dist, idx = tree.query(target)

CPU times: user 397 µs, sys: 306 µs, total: 703 µs
Wall time: 450 µs


In [7]:
print('index', idx)
print('distance', dist)
print('point', data[idx])
print()

index 57025
distance 0.7207920347972745
point [ 75.13807259 172.7460029 ]



### A right way, [3d geocentric coordinates](https://en.wikipedia.org/wiki/Earth-centered,_Earth-fixed_coordinate_system)

**So far**, we measured the euclidian distance using coordinates which is wrong! Let's consider converting coordinates to a flat space:

* Solves the distance measures inaccuracy and nearest neighbor point errors
* Point are converted from lat, lng to (x, y, z) 3d corrdinates, and those are used in the spatial index instead
* Not part of this presentation. Have a look at the link above for more details.

<img src="img/geocentric.png" width="600px"/>

### A right way, Spatial Indexing with [BallTrees](https://en.wikipedia.org/wiki/Ball_tree)

* O(log(n)) time complexity for queries
* Distance is measured correctly if correct distance measure is used (algorithm adapts to your distance measure)
* Nearest points are detected correctly without need for corrections
* Used by sklearn's nearest neigbor algorithems. Have a look at the 'algorithm' parameter in [sklearn's KNN](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)
* https://www.astroml.org/book_figures/chapter2/fig_balltree_example.html


* **Main intiuition: If I am closer to a point than the edge of a circle, then I am closer to that point than any to any point in the circle!**
* **UNLESS that point is inside the cricle** 
<img src="img/balltree.png" />

In [8]:
from sklearn.neighbors import BallTree
import numpy as np

In [9]:
%%time
bt = BallTree(np.deg2rad(data), metric='haversine')

CPU times: user 78.1 ms, sys: 3.45 ms, total: 81.6 ms
Wall time: 80.1 ms


In [10]:
%%time
dist, idx = bt.query(np.deg2rad([target]))

CPU times: user 887 µs, sys: 387 µs, total: 1.27 ms
Wall time: 741 µs


In [11]:
print('index', idx)
print('distance', dist * 6371, 'km')
print('point', data[idx])
print()

index [[87940]]
distance [[26.38197071]] km
point [[[ 74.45833535 171.93038076]]]



## Geopandas

* Geopandas relies on other libraries for geoindexing, geometry, projections, etc.
* This leads to some restrictions in features of geopandas (at the time of writing, geopandas does not deal with 3d geocentric corrdinate systems well)
* Often, KDtree from scipy or other geometric indexing tools are used to filter geopandas dataframers

## Uber's H3

* See H3 notebook